In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [37]:
# both_diff = (((df_raw["Q1"] == -1) & (df_raw["Q2"] == 1) ) | ((df_raw["Q1"] == 1) & (df_raw["Q2"] == -1))).sum()
# both_neg = ((df_raw["Q1"] == -1) & (df_raw["Q2"] == -1)).sum()
# both_pos = ((df_raw["Q1"] == 1) & (df_raw["Q2"] == 1)).sum()

# both_diff, both_neg, both_pos

In [38]:
df_cleaned = main_cleaning(df_raw)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["q1"] == df["q2"]


In [ ]:
df_cleaned

In [45]:
#df_cleaned.to_csv("data/dielectron_cleaned.csv", index=False)

In [40]:
# df_raw["prueba_e1"] = (df_raw["px1"]**2 + df_raw["py1"]**2 + df_raw["pz1"]**2)**(1/2)

In [41]:
# df_raw[df_raw["prueba_e1"].between(df_raw["E1"]*0.9999, df_raw["E1"]*1.0001)]

In [42]:
# df_raw["prueba_e2"] = (df_raw["px2"]**2 + df_raw["py2"]**2 + df_raw["pz2"]**2)**(1/2)

In [43]:
# df_raw[df_raw["prueba_e2"].between(df_raw["E2"]*0.9999, df_raw["E2"]*1.0001)]

In [44]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(15, 12))
# sns.heatmap(df_raw.corr(), cmap="Greens", annot=True)